<a href="https://colab.research.google.com/github/Bhuvaneswarij/gitingest/blob/main/Time%20series%20method1_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:


import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow.keras import layers, models
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Try prophet import (optional)
try:
    from prophet import Prophet
    PROPHET_AVAILABLE = True
except Exception as e:
    PROPHET_AVAILABLE = False
    _prophet_error = str(e)

# ----------------------
# 1) Load dataset
# ----------------------
data = sm.datasets.macrodata.load_pandas().data
# macrodata columns include: year, quarter, realgdp, realcons, realinv, realgovt, realdpi, cpi, m1, tbilrate, unemployment, pop
# convert to a datetime index (quarterly)
dates = pd.PeriodIndex(year=data['year'].astype(int), quarter=data['quarter'].astype(int), freq='Q').to_timestamp()
df = data.copy()
df.index = dates
df = df[['realgdp','realcons','realinv','realgovt','realdpi','cpi','unemp']]  # selected multivariate
df = df.rename(columns={'unemp':'unemployment'})  # nicer name
df = df.sort_index()
print("Dataset loaded: ", df.shape)
print(df.head())

# ----------------------
# 2) Expanded feature engineering
# ----------------------
def add_features(df):
    df_fe = df.copy()
    # percent changes
    df_fe[['gdp_pct','cons_pct','inv_pct']] = df_fe[['realgdp','realcons','realinv']].pct_change()
    # lags for target (unemployment) and main predictors
    for lag in [1,2,3,4]:
        df_fe[f'unemp_lag{lag}'] = df_fe['unemployment'].shift(lag)
        df_fe[f'gdp_lag{lag}'] = df_fe['realgdp'].shift(lag)
    # rolling stats
    df_fe['unemp_roll_mean_4'] = df_fe['unemployment'].rolling(window=4).mean()
    df_fe['unemp_roll_std_4'] = df_fe['unemployment'].rolling(window=4).std()
    # seasonality: quarter dummies (one-hot)
    df_fe['quarter'] = df_fe.index.quarter
    quarter_dummies = pd.get_dummies(df_fe['quarter'], prefix='q', drop_first=True)
    df_fe = pd.concat([df_fe, quarter_dummies], axis=1)
    df_fe = df_fe.drop(columns=['quarter'])
    df_fe = df_fe.dropna()
    return df_fe

df_fe = add_features(df)
print("After feature engineering:", df_fe.shape)

# ----------------------
# 3) Train/test split & scaling
# ----------------------
TARGET = 'unemployment'
feature_cols = [c for c in df_fe.columns if c != TARGET]
X_all = df_fe[feature_cols].values
y_all = df_fe[TARGET].values.reshape(-1,1)

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X_all)
y_scaled = scaler_y.fit_transform(y_all)

# create sequences for LSTM
SEQ_LEN = 8  # use 2 years of quarterly history by default (8 quarters)
def create_sequences(X, y, seq_len=SEQ_LEN):
    Xs, ys = [], []
    for i in range(seq_len, len(X)):
        Xs.append(X[i-seq_len:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys).squeeze()

X_seq, y_seq = create_sequences(X_scaled, y_scaled, SEQ_LEN)
# split
train_size = int(0.75 * len(X_seq))
X_train, X_test = X_seq[:train_size], X_seq[train_size:]
y_train, y_test = y_seq[:train_size], y_seq[train_size:]
print("Train/test shapes:", X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# helper to invert scaling
def inv_y(y_scaled_vector):
    y_scaled_vector = np.array(y_scaled_vector).reshape(-1,1)
    return scaler_y.inverse_transform(y_scaled_vector).flatten()

# ----------------------
# 4) Build MC Dropout LSTM (point + UQ via MC sampling)
# ----------------------
def build_mc_lstm(input_shape, dropout_rate=0.2):
    inp = layers.Input(shape=input_shape)
    x = layers.LSTM(64, return_sequences=True)(inp)
    x = layers.Dropout(dropout_rate)(x, training=True)  # ensure dropout active for MC sampling
    x = layers.LSTM(32)(x)
    x = layers.Dropout(dropout_rate)(x, training=True)
    out = layers.Dense(1)(x)
    model = models.Model(inp, out)
    model.compile(optimizer='adam', loss='mse')
    return model

mc_model = build_mc_lstm((SEQ_LEN, X_train.shape[2]), dropout_rate=0.2)
mc_model.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0)
print("MC LSTM trained.")

# MC prediction function (keep dropout active)
def mc_predict(model, X, sims=200):
    preds = []
    for i in range(sims):
        p = model(X, training=True).numpy().flatten()
        preds.append(p)
    preds = np.array(preds)  # shape (sims, n_samples)
    return preds

# ----------------------
# 5) Quantile Regression LSTM
# ----------------------
# We'll predict quantiles: 0.025, 0.10, 0.5, 0.9, 0.975  -> use these to form 80% (10-90) and 95% (2.5-97.5)
quantiles = [0.025, 0.10, 0.5, 0.9, 0.975]
def quantile_loss(q):
    def loss(y_true, y_pred):
        e = y_true - y_pred
        return tf.reduce_mean(tf.maximum(q*e, (q-1)*e))
    return loss

def build_quantile_lstm(input_shape):
    inp = layers.Input(shape=input_shape)
    x = layers.LSTM(64)(inp)
    # one dense per quantile
    outputs = [layers.Dense(1, name=f'q_{int(q*1000)}')(x) for q in quantiles]
    model = models.Model(inp, outputs)
    model.compile(optimizer='adam', loss=[quantile_loss(q) for q in quantiles])
    return model

q_model = build_quantile_lstm((SEQ_LEN, X_train.shape[2]))
# prepare y repeated per output
y_train_list = [y_train for _ in quantiles]
q_model.fit(X_train, y_train_list, epochs=40, batch_size=16, verbose=0)
print("Quantile LSTM trained.")

# ----------------------
# 6) Classical benchmarks: SARIMAX (on unemployment series) and Prophet (if available)
# ----------------------
# SARIMAX on the original unemployment series aligned with sequences' last indices.
unemp_series = df_fe[TARGET]
# For SARIMAX forecast we need aligned index range: we will use last len(y_test) observations as hold-out
sarimax_train_end = df_fe.index[train_size + SEQ_LEN - 1]  # end date of training period
sar_model = SARIMAX(unemp_series[:sarimax_train_end], order=(1,1,1), seasonal_order=(0,0,0,0), enforce_stationarity=False, enforce_invertibility=False)
sar_res = sar_model.fit(disp=False)
sar_forecast = sar_res.get_forecast(steps=len(y_test))
sar_pred = sar_forecast.predicted_mean.values
sar_ci = sar_forecast.conf_int(alpha=0.05)  # 95% CI from SARIMAX (if available)

# Prophet (optional): we must assemble ds,y dataframe
prophet_pred = None
prophet_df = None
if PROPHET_AVAILABLE:
    prop_df = pd.DataFrame({'ds': df_fe.index, 'y': df_fe[TARGET].values})
    prophet = Prophet()
    prophet.fit(prop_df[:len(prop_df)-len(y_test)])
    future = prophet.make_future_dataframe(periods=len(y_test), freq='Q')
    forecast = prophet.predict(future)
    prophet_pred = forecast['yhat'].values[-len(y_test):]
    prophet_lower95 = forecast['yhat_lower'].values[-len(y_test):]
    prophet_upper95 = forecast['yhat_upper'].values[-len(y_test):]
else:
    print("Prophet is not available in this environment. To enable Prophet, install with 'pip install prophet' and rerun.")
    print("Prophet import error (first lines):", _prophet_error[:200])

# ----------------------
# 7) Predictions + UQ
# ----------------------
# MC-LSTM preds
mc_sims = 300
mc_preds_scaled = mc_predict(mc_model, X_test, sims=mc_sims)  # shape (sims, n_samples)
# derive percentiles for 80% and 95%
mc_lower_80 = np.percentile(mc_preds_scaled, 10, axis=0)
mc_upper_80 = np.percentile(mc_preds_scaled, 90, axis=0)
mc_lower_95 = np.percentile(mc_preds_scaled, 2.5, axis=0)
mc_upper_95 = np.percentile(mc_preds_scaled, 97.5, axis=0)
mc_mean_scaled = mc_preds_scaled.mean(axis=0)

# invert scaling
mc_mean = inv_y(mc_mean_scaled)
mc_l80 = inv_y(mc_lower_80)
mc_u80 = inv_y(mc_upper_80)
mc_l95 = inv_y(mc_lower_95)
mc_u95 = inv_y(mc_upper_95)

# Quantile LSTM preds
q_preds_scaled = q_model.predict(X_test)  # list of arrays -> one per quantile
# ordering matches quantiles list
q_dict = {}
for i, q in enumerate(quantiles):
    q_dict[q] = q_preds_scaled[i].flatten()

# invert
q_inv = {q: inv_y(q_dict[q]) for q in quantiles}
q_median = q_inv[0.5]
q_l80 = q_inv[0.10]
q_u80 = q_inv[0.90]
q_l95 = q_inv[0.025]
q_u95 = q_inv[0.975]

# SARIMAX invert -> sar_pred already in original scale because sarimax trained on original series
sar_pred = sar_pred  # array in original units
# For SARIMAX we may not have 80% intervals; we will compute 95% from sar_ci if available; compute 80% approx by widening/assuming normality
if 'lower unemp' in sar_ci.columns or 'lower unemployment' in sar_ci.columns:
    pass
# But sar_ci columns will be like ['lower y', 'upper y'] for the variable; handle generically
if sar_ci.shape[1] == 2:
    sar_lower95 = sar_ci.iloc[:,0].values
    sar_upper95 = sar_ci.iloc[:,1].values
    # approximate 80% by linear interpolation between mean and 95% boundaries (not ideal, but gives an estimate)
    sar_lower80 = sar_lower95 * 0.65 + sar_pred * 0.35
    sar_upper80 = sar_upper95 * 0.65 + sar_pred * 0.35
else:
    sar_lower95 = np.full_like(sar_pred, np.nan)
    sar_upper95 = np.full_like(sar_pred, np.nan)
    sar_lower80 = np.full_like(sar_pred, np.nan)
    sar_upper80 = np.full_like(sar_pred, np.nan)

# Prophet output (already in original scale) if available
if PROPHET_AVAILABLE:
    prop_mean = prophet_pred
    prop_l95 = prophet_lower95
    prop_u95 = prophet_upper95
    # attempt 80% by interpolation similar to SARIMAX (or compute 10th/90th if Prophet provided—Prophet gives 80% if you request)
    prop_l80 = prop_l95 * 0.65 + prop_mean * 0.35
    prop_u80 = prop_u95 * 0.65 + prop_mean * 0.35

# ----------------------
# 8) Evaluation metrics (point predictions)
# ----------------------
def mae(y_true, y_pred): return mean_absolute_error(y_true, y_pred)
def rmse(y_true, y_pred): return np.sqrt(mean_squared_error(y_true, y_pred))
def mape(y_true, y_pred): return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# true y_test in original scale
y_true = inv_y(y_test)

results = {}

# MC LSTM
results['MC LSTM'] = {
    'MAE': mae(y_true, mc_mean),
    'RMSE': rmse(y_true, mc_mean),
    'MAPE': mape(y_true, mc_mean)
}

# Quantile median
results['Quantile LSTM (median)'] = {
    'MAE': mae(y_true, q_median),
    'RMSE': rmse(y_true, q_median),
    'MAPE': mape(y_true, q_median)
}

# SARIMAX
results['SARIMAX'] = {
    'MAE': mae(y_true, sar_pred),
    'RMSE': rmse(y_true, sar_pred),
    'MAPE': mape(y_true, sar_pred)
}

# Prophet (if present)
if PROPHET_AVAILABLE:
    results['Prophet'] = {
        'MAE': mae(y_true, prop_mean),
        'RMSE': rmse(y_true, prop_mean),
        'MAPE': mape(y_true, prop_mean)
    }

# ----------------------
# 9) Uncertainty metrics: ICP and MIW for each model and both intervals (80% & 95%)
# ----------------------
def interval_metrics(y_true, lower, upper):
    y_true = np.array(y_true)
    lower = np.array(lower)
    upper = np.array(upper)
    # remove NaNs pairs
    mask = ~(np.isnan(lower) | np.isnan(upper))
    if mask.sum() == 0:
        return {'ICP': np.nan, 'MIW': np.nan}
    covered = (y_true[mask] >= lower[mask]) & (y_true[mask] <= upper[mask])
    ICP = covered.mean()
    MIW = np.mean(upper[mask] - lower[mask])
    return {'ICP': ICP, 'MIW': MIW, 'n': mask.sum()}

uq_results = {}

# MC LSTM intervals
uq_results['MC LSTM 80%'] = interval_metrics(y_true, mc_l80, mc_u80)
uq_results['MC LSTM 95%'] = interval_metrics(y_true, mc_l95, mc_u95)

# Quantile LSTM intervals
uq_results['Quantile LSTM 80%'] = interval_metrics(y_true, q_l80, q_u80)
uq_results['Quantile LSTM 95%'] = interval_metrics(y_true, q_l95, q_u95)

# SARIMAX intervals
uq_results['SARIMAX 80%'] = interval_metrics(y_true, sar_lower80, sar_upper80)
uq_results['SARIMAX 95%'] = interval_metrics(y_true, sar_lower95, sar_upper95)

# Prophet intervals if available
if PROPHET_AVAILABLE:
    uq_results['Prophet 80%'] = interval_metrics(y_true, prop_l80, prop_u80)
    uq_results['Prophet 95%'] = interval_metrics(y_true, prop_l95, prop_u95)

# ----------------------
# 10) Print summary results
# ----------------------
print("\n=== POINT FORECAST METRICS ===")
for model, r in results.items():
    print(f"{model}: MAE={r['MAE']:.4f}, RMSE={r['RMSE']:.4f}, MAPE={r['MAPE']:.2f}%")

print("\n=== UNCERTAINTY METRICS (ICP, MIW) ===")
for k, v in uq_results.items():
    print(f"{k}: ICP={v['ICP']}, MIW={v['MIW']}, n={v.get('n', None)}")

# ----------------------
# 11) Detailed text-based analysis report (prints)
# ----------------------
def print_analysis(results, uq_results):
    print("\n\n==================== DETAILED ANALYSIS REPORT ====================\n")
    print("Dataset:")
    print(f" - Source: statsmodels.datasets.macrodata (quarterly)")
    print(f" - Features used: {feature_cols}")
    print(f" - Sequence length (history): {SEQ_LEN} quarters")
    print("\nPoint Forecast Comparison:")
    for m, r in results.items():
        print(f" - {m}: MAE={r['MAE']:.4f}, RMSE={r['RMSE']:.4f}, MAPE={r['MAPE']:.2f}%")
    print("\nUncertainty (Prediction Interval) Comparison:")
    for model_name in ['MC LSTM 80%','MC LSTM 95%','Quantile LSTM 80%','Quantile LSTM 95%','SARIMAX 80%','SARIMAX 95%']:
        if model_name in uq_results:
            v = uq_results[model_name]
            print(f" - {model_name}: ICP={v['ICP']:.3f}, MIW={v['MIW']:.3f} (n={v.get('n',None)})")
    if PROPHET_AVAILABLE:
        v = uq_results.get('Prophet 95%')
        if v:
            print(f" - Prophet 95%: ICP={v['ICP']:.3f}, MIW={v['MIW']:.3f}")
    print("\nInterpretation guidelines:")
    print(" - ICP (Interval Coverage Probability): fraction of true values lying inside the PI. For an 80% PI target ~0.8, for a 95% PI target ~0.95.")
    print(" - MIW (Mean Interval Width): average width of the PIs; narrower is better if ICP is near nominal target. Trade-off: too narrow -> low ICP; too wide -> less useful.")
    print("\nModel strengths / weaknesses (expected):")
    print(" - MC LSTM: flexible, captures complex nonlinearities; MC Dropout provides model-driven epistemic uncertainty. Check whether ICPs are near nominal levels.")
    print(" - Quantile LSTM: can directly optimize different quantiles; tends to produce sharper intervals when trained well.")
    print(" - SARIMAX: classical, may be competitive for linear seasonal series; intervals are parametric and rely on model assumptions.")
    if PROPHET_AVAILABLE:
        print(" - Prophet: robust for trend+seasonality and business time series; uncertainty is based on simulated components and may be wider/narrower depending on trend assumptions.")
    print("\nPractical next steps:")
    print(" 1) If ICP significantly below nominal -> increase model capacity or use ensembling/heteroscedastic modeling.")
    print(" 2) If MIW too large -> retrain quantile model with stronger regularization or add relevant predictors.")
    print(" 3) Cross-validate interval calibration using rolling-window CV and adjust quantile calibration post-hoc if necessary.")
    print("\n=================================================================\n")

print_analysis(results, uq_results)


Dataset loaded:  (203, 7)
             realgdp  realcons  realinv  realgovt  realdpi    cpi  \
1959-01-01  2710.349    1707.4  286.898   470.045   1886.9  28.98   
1959-04-01  2778.801    1733.7  310.859   481.301   1919.7  29.15   
1959-07-01  2775.488    1751.8  289.226   491.260   1916.4  29.35   
1959-10-01  2785.204    1753.7  299.356   484.052   1931.3  29.37   
1960-01-01  2847.699    1770.5  331.722   462.199   1955.5  29.54   

            unemployment  
1959-01-01           5.8  
1959-04-01           5.1  
1959-07-01           5.3  
1959-10-01           5.6  
1960-01-01           5.2  
After feature engineering: (199, 23)
Train/test shapes: (143, 8, 22) (48, 8, 22) (143,) (48,)
MC LSTM trained.


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Quantile LSTM trained.
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 292ms/step

=== POINT FORECAST METRICS ===
MC LSTM: MAE=1.1065, RMSE=1.3170, MAPE=20.08%
Quantile LSTM (median): MAE=1.3504, RMSE=1.5293, MAPE=25.03%
SARIMAX: MAE=0.8210, RMSE=1.2710, MAPE=13.83%
Prophet: MAE=1.1127, RMSE=1.4634, MAPE=22.24%

=== UNCERTAINTY METRICS (ICP, MIW) ===
MC LSTM 80%: ICP=0.125, MIW=1.0116974115371704, n=48
MC LSTM 95%: ICP=0.2916666666666667, MIW=1.515511393547058, n=48
Quantile LSTM 80%: ICP=0.3125, MIW=0.754157543182373, n=48
Quantile LSTM 95%: ICP=0.875, MIW=1.8983834981918335, n=48
SARIMAX 80%: ICP=1.0, MIW=7.3749660714157175, n=48
SARIMAX 95%: ICP=1.0, MIW=11.346101648331873, n=48
Prophet 80%: ICP=0.5416666666666666, MIW=2.003576122927335, n=48
Prophet 95%: ICP=0.6458333333333334, MIW=3.0824248045035922, n=48


==================== DETAILED ANALYSIS REPORT ====================

Dataset:
 - Source: statsmodels.datasets.macrodata (quarterly)
 - Features used: ['realgdp', 'realcons', 'realinv', 'realgovt', 